In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.applications import VGG16

In [3]:
conv_base = VGG16(weights = 'imagenet',
                 include_top=False,
                 input_shape=(150, 150, 3))

58892288/58889256 [==============================] - 1s 0us/step


In [4]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [19]:
base_dir = '/home/ec2-user/SageMaker/all'

train_dir = os.path.join(base_dir,'train')
#validation_dir = os.path.join(base_dir,'validation')
test_dir = os.path.join(base_dir,'test1')

datagen = ImageDataGenerator(rescale=1/255)
batch_size= 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count,4,4,512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(directory,
                                           target_size=(150,150),
                                           batch_size=batch_size,
                                           class_mode='binary')
    i = 0
    for inputs_batch,labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i+1) * batch_size] = features_batch
        labels[i * batch_size: (i+1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must 'break' after every image has been seen once.
            break
    return features, labels

train_features,train_labels = extract_features(train_dir,2000)
#validation_features, validation_labels = extract_features(validation_dir,1000)
test_features, test_labels = extract_features(test_dir,1000)
    

Found 0 images belonging to 0 classes.


ZeroDivisionError: integer division or modulo by zero

In [14]:
# Flatten it out first
train_features = np.reshape(train_features,(2000, 4*4*512))
#validation_features = np.reshape(validation_features,(1000, 4*4*512))
test_features = np.reshape(test_features,(1000, 4*4*512))

NameError: name 'train_features' is not defined

In [ ]:
# importing layers, moels and optimizers
from keras import models
from keras import layers
from keras import optimizers

# construct model
model = models.Sequential()
model.add(layers,Dense(256,activation='relu',input_dim = 4*4*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# compile model

model.compile(optimizer = optimizers.RMSprop(1r=2e-5),
             loss = 'binary_crossentropy',
             metrics = ['acc'])

# Run the model

history = model.fit(train_features,train_labels,
                    epochs = 30,
                    batch_size = 20,
                    validation_data = (validation_features, validation_labels))
